In [1]:
import tensorflow as tf
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None),name="y")


training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",reuse=False)
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", reuse=False)
bn2 =  tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)
hidden2_stop = tf.stop_gradient(hidden2)

hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3", reuse=False)
bn3 =  tf.layers.batch_normalization(hidden3, training=training, momentum=0.9)
bn3_act = tf.nn.elu(bn3)
logits_before_bn = tf.layers.dense(bn3_act,n_outputs, name="outputs", reuse=False)
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)






with tf.name_scope("loss"):
    #equivalent with applying solfmax then computing cross_entr
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                             logits=logits)
    loss = tf.reduce_mean(xentropy,name="loss")


learning_rate = 0.01

with tf.name_scope("train"):
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    ##we can freeze lower layers by training only the last 

    #train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              #scope="hidden[3]|outputs")
    #training_op = optimizer.minimize(loss, var_list=train_vars)
    training_op = optimizer.minimize(loss)
    

    
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

W0828 14:11:05.918681 140226947921728 deprecation.py:323] From <ipython-input-1-15ffee5049b0>:13: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0828 14:11:05.920945 140226947921728 deprecation.py:506] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0828 14:11:06.122006 140226947921728 deprecation.py:323] From <ipython-input-1-15ffee5049b0>:14: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKey

In [2]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                              scope="hidden[12]")

reuse_vars_dict = dict([var.op.name, var] for var in reuse_vars)
restore_saver = tf.train.Saver(reuse_vars_dict)#to restrore layers

init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

W0828 14:11:07.709379 140226947921728 deprecation.py:323] From <ipython-input-3-c50d5bb4a85c>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0828 14:11:07.710105 140226947921728 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0828 14:11:07.711004 140226947921728 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
I

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0828 14:11:07.915434 140226947921728 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0828 14:11:07.957413 140226947921728 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:


extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

  
n_epochs = 20
batch_size = 50
with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Models/another_digir.ckpt")
    #[...] train the model
    for epoch in range(n_epochs):
        for iteraion in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops],
                    feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                               y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

        
    save_path = saver.save(sess, "Models/yet_another.ckpt")

W0828 14:11:08.741912 140226947921728 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) Not found: Key hidden1/bias/Momentum not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-2-ed784a5f8e8e>:5) ]]
	 [[save/RestoreV2/_11]]
  (1) Not found: Key hidden1/bias/Momentum not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-2-ed784a5f8e8e>:5) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "/root/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_insta

In [ ]:
#future research : Unsupervsed Pretraining